### 1. 한글 원본 텍스트를 자소 단위와 단어 단위로 나누어서 순환 신경망으로 생성해 보고자 한다.

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
df = open('/content/sample_data/california_housing_train.csv', 'rb').read().decode(encoding = 'utf-8')
df = df.split('\n')

In [3]:
df[1].split(',')[0]

'-114.310000'

In [4]:
labels = []
for i in df[0].split(','):
  labels.append(i)

In [5]:
file = []
for i in df:
  file.append(i.split(','))

In [6]:
file[len(file)-2]

['-124.350000',
 '40.540000',
 '52.000000',
 '1820.000000',
 '300.000000',
 '806.000000',
 '270.000000',
 '3.014700',
 '94600.000000']

In [7]:
d = dict()
for i in range(len(labels)):
  temp = []
  for k in range(1,len(file)-1):
    temp.append(file[k][i])
  d[labels[i]] = temp

In [8]:
data = pd.DataFrame(d)

In [9]:
data.head()

,"""longitude""","""latitude""","""housing_median_age""","""total_rooms""","""total_bedrooms""","""population""","""households""","""median_income""","""median_house_value"""
0,-114.310000,34.190000,15.000000,5612.000000,1283.000000,1015.000000,472.000000,1.493600,66900.000000
1,-114.470000,34.400000,19.000000,7650.000000,1901.000000,1129.000000,463.000000,1.820000,80100.000000
2,-114.560000,33.690000,17.000000,720.000000,174.000000,333.000000,117.000000,1.650900,85700.000000
3,-114.570000,33.640000,14.000000,1501.000000,337.000000,515.000000,226.000000,3.191700,73400.000000
4,-114.570000,33.570000,20.000000,1454.000000,326.000000,624.000000,262.000000,1.925000,65500.000000


- 아무 데이터나 이용해 보았는데, csv파일 또한 **파이썬의 파일 입출력 기능인 open으로 이용이 가능**하다(저장 경로만 알고 있으면)
  - 그런데 pandas dataframe의 경우에는 dict파일로 변환된 자료를 바로 바꿀 수 있고, 아니라면 
    1. DataFrame.from_records
      :Constructor from tuples, also record arrays.

    2. DataFrame.from_dict
      :From dicts of Series, arrays, or dicts.

    3. read_csv
      :Read a comma-separated values (csv) file into DataFrame.

    4. read_table
      :Read general delimited file into DataFrame.

    5. read_clipboard
        :Read text from clipboard into DataFrame.

    이 5가지 방법도 존재한다.

-데이터를 다룰떄 자연어 처리같은 작업이 아니라면 저렇게 형태를 바꿔주면 좋다.

- 파이썬의 open 기능은 'r', 'w','rd'등 다양한데, 순서대로 읽기, 쓰기, 이진 파일로 받기 등의 방법이다.
  - 그러나 일단 텍스트 파일로 받게 된다.

#### 1. 데이터 로드

In [10]:
path = tf.keras.utils.get_file('input.txt', 'http://bit.ly/2Mc3SOV')

62013440/62012502 [==============================] - 1s 0us/step


In [11]:
#데이터를 메모리에 불러오는데 우선 인코딩 형식으로 'utf-8'을 사용하기로 한다.
train_text = open(path, 'rb').read().decode(encoding = 'utf-8')

In [12]:
train_text[:100]

'\ufeff태조 이성계 선대의 가계. 목조 이안사가 전주에서 삼척·의주를 거쳐 알동에 정착하다 \n태조 강헌 지인 계운 성문 신무 대왕(太祖康獻至仁啓運聖文神武大王)의 성은 이씨(李氏)요, 휘'

In [13]:
#데이터 정제를 위한 함수 만들기
#우선 영문은 없고 한글만 있을 것이기 떄문에(조선 왕조 실록 데이터임)
import re

def clean_str(string):
  string = re.sub(r'[^가-힣A-Za-z0-9(),!?\'\`]', ' ', string)
  string = re.sub(r'\'ll',' \ll',string)
  string = re.sub(r',', ' , ',string)
  string = re.sub(r'!', ' ! ',string)
  string = re.sub(r'\?', ' ? ',string)
  string = re.sub(r'\)', '',string)
  string = re.sub(r'\(','',string)
  string = re.sub(r'\s{2,}',' ',string)
  string = re.sub(r'\`','',string)
  string = re.sub(r'\'{2,}','\'',string)

  return string

In [14]:
train_text = train_text.split('\n')  #줄별로 띄어서 인식
train_text = [clean_str(i) for i in train_text]
train_x = []
for i in train_text:
  train_x.extend(i.split(' '))
  train_x.append('\n')


In [15]:
train_x = [i for i in train_x if i != '']
train_x[:20]

['태조',
 '이성계',
 '선대의',
 '가계',
 '목조',
 '이안사가',
 '전주에서',
 '삼척',
 '의주를',
 '거쳐',
 '알동에',
 '정착하다',
 '\n',
 '태조',
 '강헌',
 '지인',
 '계운',
 '성문',
 '신무',
 '대왕']

#### 2. 단어 토큰화
- 한가지 헷갈리고 있었던 사실은 토큰화라는 것이 단어 벡터를 형성해 주는 것이라고 생각했던 것이다. 
  - 하지만 그것은 원핫 인코딩과 임베딩층에 의해 진행되는 것이다.
  - 우리가 자연어 처리를 위해 데이터를 크롤링하거나 수집했을 떄 목적에 맞게 바꿔줄 필요가 있고, 이를 데이터 전처리라고 한다.
    - 그 예시로 Tokenization, Cleaning, Normalization이 있는데 Cleaning은 말 그대로 불필요한 문자 등을 문장/문자열 단위로 제거해주는 것이고, Tokenization은 우리가 사용할 의미있는 단위별로 나누어서 구성하는 것을 의미한다.

In [16]:
#단어의 set를 만든다
#모든 데이터의 단어들을 입력하다 보니 겹치는 것이 있을 수 있어 그것을 없애주기 위해서 set에 넣는다.
word = sorted(set(train_x))
word.append('UNK')

#단어 리스트를 숫자로 매핑하고, 반대도 실행한다.
word2idx = {u:i for i,u in enumerate(word)}
idx2word = np.array(word)

text_as_int = np.array([word2idx[i] for i in train_x])

print(text_as_int[:10])


[299333 229662 161471  17456 111055 230320 251109 155115 225490  29053]


- 위의 출력된 값을 보면 알 수 있듯이 우리는 데이터의 토큰화를 마무리 했다.
- 실제로 ```from tensorflow.keras.preprocessing.text import Tokenizer```을 통해서(저 모듈을 이용하면 정해주는 단어의 개수만큼의 빈도수별로 정렬된 저장소에서 단어를 찾아 인덱싱을 한다) 토큰화를 진행했어도 상관 없었겠지만 이 데이터의 경우 단어도 너무 많기 때문에 시간 대비 효율적이지 못하다는 판단이 든다.

- 'UNK'라는 값을 삽입한 이유는 나중에 학습을 할 때 저장되지 않은 단어를 위한 자리이다.

In [17]:
len(text_as_int), len(word)

(6834919, 332669)

- 총 사용된 단어의 개수는(중복 없이) 332669개이지만 중복을 포함한다면 전체 데이처에 담긴 토큰화된 단어의 개수는 6834919개이다.

In [18]:
print(train_x[:20])
print(text_as_int[:20])

['태조', '이성계', '선대의', '가계', '목조', '이안사가', '전주에서', '삼척', '의주를', '거쳐', '알동에', '정착하다', '\n', '태조', '강헌', '지인', '계운', '성문', '신무', '대왕']
[299333 229662 161471  17456 111055 230320 251109 155115 225490  29053
 190323 256157      0 299333  25650 273581  36173 164024 180494  84439]


- 개행 문자인 '\n'는 0으로 변환이 되었음을 알 수 있다.

#### 3. 학습을 위한 데이터셋 만들기
- 여기서 학습의 목적을 다시 상기해 보자
  - 우리는 일정한 길이의 부분 문장이 주어졌을 때 그 데이터를 완성시킬 수 있도록 학습을 해야 한다.
  - 그렇기 때문에 학습 데이터를 만들 때에도 일정 길이의 문장을 학습 데이터로 주고 이어지는 부분을 label,즉 y값으로 입력을 함으로서 학습을 진행해야 한다.

In [19]:
#우리는 한 데이터당(x의 데이터) 문장의 길이를 seq_length라는 변수를 25로 지정해 주는 방법으로 제한하고자 한다.
seq_length = 25
#batch_size를 설정했을 때 steps_per_epoch를 전체 데이터의 길이에서 batch_size로 나누는것과 같은 상황이다.
examples_per_epoch = len(text_as_int) // seq_length
sentence_data = tf.data.Dataset.from_tensor_slices(text_as_int)

- 위와 같이 tensor dataset로 이루어진 데이터의 경우에는 하나씩 골라내어서 numpy()의 형태로 바꿔준 뒤에 어떤 값인지 봐야 한다.

In [20]:
for i in sentence_data.take(10):
  print(i)
  print(i.numpy())

tf.Tensor(299333, shape=(), dtype=int64)
299333
tf.Tensor(229662, shape=(), dtype=int64)
229662
tf.Tensor(161471, shape=(), dtype=int64)
161471
tf.Tensor(17456, shape=(), dtype=int64)
17456
tf.Tensor(111055, shape=(), dtype=int64)
111055
tf.Tensor(230320, shape=(), dtype=int64)
230320
tf.Tensor(251109, shape=(), dtype=int64)
251109
tf.Tensor(155115, shape=(), dtype=int64)
155115
tf.Tensor(225490, shape=(), dtype=int64)
225490
tf.Tensor(29053, shape=(), dtype=int64)
29053


In [21]:
sentence_data = sentence_data.batch(seq_length + 1, drop_remainder = True)
for i in sentence_data.take(1):
  print(idx2word[i.numpy()])
  print(i.numpy())

['태조' '이성계' '선대의' '가계' '목조' '이안사가' '전주에서' '삼척' '의주를' '거쳐' '알동에' '정착하다'
 '\n' '태조' '강헌' '지인' '계운' '성문' '신무' '대왕' '의' '성은' '이씨' '요' ',' '휘']
[299333 229662 161471  17456 111055 230320 251109 155115 225490  29053
 190323 256157      0 299333  25650 273581  36173 164024 180494  84439
 224210 164577 230276 210940     28 330342]


In [22]:
for i in sentence_data.take(1):
  print(i)

tf.Tensor(
[299333 229662 161471  17456 111055 230320 251109 155115 225490  29053
 190323 256157      0 299333  25650 273581  36173 164024 180494  84439
 224210 164577 230276 210940     28 330342], shape=(26,), dtype=int64)


- 위에서 tf.data.Dataset으로 텐서의 형태로 바꾸어 주었는데, Dataset에 쓰이는 batch() 함수는 Dataset에서 한번에 반환하는 데이터의 숫자를 지정해 준다.
- 그리고 여기서 seq_length+1로 지정을 해 주었는데, 이는 마지막에 정답이 될 1개의 단어를 합쳐서 반환하기 위해서이다.

- 이제 이 Dataset으로 ([25개의 단어], 1개의 단어)이런 형태의 데이터로 바뀐 Dataset을 만들어 주어야 한다.

In [32]:
def split_input_data(data):
  return [data[:-1], data[-1]]

dataset = sentence_data.map(split_input_data)
for i in dataset.take(1):
  print(i[0].numpy())  #학습을 입력받는 데이터
  print(i[1].numpy())  #학습의 예측의 결과가 되어야 하는 값

[299333 229662 161471  17456 111055 230320 251109 155115 225490  29053
 190323 256157      0 299333  25650 273581  36173 164024 180494  84439
 224210 164577 230276 210940     28]
330342


In [33]:
batch_size = 600
steps_per_epoch = len(dataset)//batch_size
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder = True)

- 빠른 학습을 위해서 한번에 128개의 데이터를 학습하도록 했고, 데이터를 섞을 때의 buffer_size를 10000으로 설정을 했다.

#### 4. 모델 만들기 및 학습

In [34]:
model = tf.keras.models.Sequential()
#Embedding Layer안의 변수들은 (input_dim, output_dim, input_length) 이다.
model.add(tf.keras.layers.Embedding(len(word), 100, input_length = 25))
model.add(tf.keras.layers.LSTM(units = 100, return_sequences=True))
model.add(tf.keras.layers.Dropout(rate = 0.2))
model.add(tf.keras.layers.LSTM(units = 100))
model.add(tf.keras.layers.Dense(len(word2idx), activation = 'softmax'))

model.summary()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 25, 100)           33266900  
_________________________________________________________________
lstm_4 (LSTM)                (None, 25, 100)           80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 25, 100)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 332669)            33599569  
Total params: 67,027,269
Trainable params: 67,027,269
Non-trainable params: 0
_________________________________________________________________


In [42]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def testmodel(epoch, logs):
  if epoch%5 != 0 and epoch != 49:
    return
  test_sentence = train_text[0]
  next_words = 100
  for _ in range(next_words):
    x_test = test_sentence.split(' ')[-seq_length:]
    x_test = np.array([word2idx[i] if i in word2idx else word2idx['UNK']for i in x_test])
    x_test = pad_sequences([x_test], maxlen = seq_length, padding = 'post', value = word2idx['UNK'])
    output = model.predict_classes(x_test)
    #그냥 predict()를 하게 되면 출력값의 개수가 전체 단어 개수인 332640개이므로 predict_classes를 이용해서 출력값이 가장 큰 인덱스를 이용
    test_sentence += ' '+idx2word[output[0]]
  print()
  print(test_sentence)
  print()

- tensor dataset으로는 validation split를 적용하기가 어려워서 직접 학습 결과를 확인하기 위해서 testmodel이라는 이름으로 callback 함수를 정의했다.
- 임의의 문장을 입력하고, 뒤에서부터 25개의 단어를 선택하고, 이 단어를 word2idx에 저장된 인덱스로 바꾸어주었다. 단, 만약에 저장이 안되어있는 단어라면 'UNK'의 인덱스를 입력하였다. 
- 그리고 입력한 25길이의 문장에 예측한 뒤의 단어를 덧붙여서 결괏값으로서 출력이 되도록 하였다.
- 출력 단어는 test_sentence의 끝부분에 저장되어 다음 스텝의 입력에 활용이 되는데 중요한 것은 학습을 시킬 때에 특히 keras dataset을 이용하기 때문에 데이터의 처음과 끝을 모르는 관계로 .repeat()를 이용해 끊임없이 반복 적용해야 한다.

In [43]:
testcallback = tf.keras.callbacks.LambdaCallback(on_epoch_end = testmodel)
history = model.fit(dataset.repeat(), epochs = 7, steps_per_epoch=steps_per_epoch, callbacks = [testcallback])

Epoch 1/7
438/438 [==============================] - ETA: 0s - loss: 8.1428 - accuracy: 0.0804WARNING:tensorflow:From <ipython-input-42-37b6b0be43b5>:12: Sequential.predict_classes (from tensorflow.python.keras.engine.sequential) is deprecated and will be removed after 2021-01-01.
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).

 태조 이성계 선대의 가계 목조 이안사가 전주에서 삼척 의주를 거쳐 알동에 정착하다  의 의 의 의 의 의 의 의 그 의 을 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


438/438 [==============================] - 249s 569ms/step - loss: 8.1428 - accuracy: 0.0804
Epoch 2/7
438/438 [======================

#### 6. 임의의 문장을 이용해서 생성 결과 확인

In [50]:
train_text[23]

' 그 짐승이 많은 것을 탐내서 지금까지 돌아오지 않습니다 '

In [49]:
sentence = train_text[23]
from tensorflow.keras.preprocessing.sequence import pad_sequences
next_word = 100
for _ in range(next_word):
  test = sentence.split(' ')[-seq_length:]
  test = [[word2idx[i] if i in word2idx else word2idx['UNK']]for i in test]
  test = pad_sequences([test], maxlen = seq_length, padding = 'post', value = word2idx['UNK'])
  pred = model.predict_classes(test)
  pred = idx2word[pred[0]]
  sentence += ' '+pred
print(sentence)


 그 짐승이 많은 것을 탐내서 지금까지 돌아오지 않습니다  의 의 의 의 의 이 이 이 이 이 이 이 한 이 이 , 그 말을 아뢰기를 , 
 하였다 임금이 말하기를 , 
 임금이 그 관찰사 을 보내어 명하여 그 주고 , 그 보내어 와서 가지고 와서 가지고 와서 가지고 와서 가지고 와서 가지고 와서 가지고 와서 가지고 와서 와서 가서 가지고 와서 와서 가서 가지고 와서 와서 가서 가지고 와서 와서 가서 가지고 와서 와서 가서 가지고 와서 와서 가서 가지고 와서 와서 가서 가지고 와서 와서 가서 가지고 와서 와서 가서 가지고 와서 와서 가서 가지고 와서 와서 가서 가지고 와서 와서 가서 가지고 와서


### 2. 자모 단위 생성
- 위에서 진행한 작업은 단어 단위로 자연어를 학습하고 생성했다면 이번에는 자모단위로 자연어를 생성하는 연습을 하고자 한다.
- 그렇게 하기 위해서는 자모 단위로 분리를 해야 하는데, 이는 이미 만들어져 있는 jamotools라는 라이브러리를 사용하고자 한다.

In [53]:
!pip install jamotools

In [55]:
import jamotools
train_text = open(path, 'rb').read().decode(encoding = 'utf-8')

train_x = jamotools.split_syllables(train_text)
word = sorted(set(train_x))
word.append('UNK')

char2idx = {u:i for i,u in enumerate(word)}
idx2char = np.array(word)

#문서의 자모를 나눈 것을 전부 인덱스로 바꾼 데이터
text_as_int = np.array([char2idx[i] if i in char2idx else char2idx['UNK']for i in train_x])



In [56]:
seq_length = 80
#한번에 80의 길이를 가진 자모 모음을 학습 입력값으로 사용할 것임
#examples_per_epoch는 한번의 epoch에 학습하는 묶음의 개수
examples_per_epoch = len(text_as_int) // seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

#데이터셋을 (x,y)꼴로, 즉 (입력할 seq_length의 길이의 자모 모음, 뒤에 이어질 자모)
char_dataset = char_dataset.batch(seq_length+1, drop_remainder = True)
#map()함수로 x와 y를 하나의 데이터로 묶어줌
char_dataset = char_dataset.map(split_input_data)

batch_size = 600
bufffer_size = 10000
char_dataset = char_dataset.shuffle(buffer_size).batch(batch_size, drop_remainder = True)
steps_per_epoch = examples_per_epoch//batch_size


In [59]:
#중복 없는 분리된 자모의 개수
total_char = len(word)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(len(word), 100, input_length = seq_length))
model.add(tf.keras.layers.LSTM(units = 100, return_sequences=  True))
model.add(tf.keras.layers.Dropout(rate = 0.3))
model.add(tf.keras.layers.LSTM(units = 100))
model.add(tf.keras.layers.Dense(len(word), activation = 'softmax'))

model.summary()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


def test_cb(epoch, logs):
  if epoch%2 != 0 and epoch != 6:
    return
  sentence = train_text[:48]
  for _ in range(300):
    test = jamotools.split_syllables(sentence)
    test = test[-seq_length:]
    test = np.array([char2idx[i] if i in char2idx else char2idx['UNK']for i in test])
    test = pad_sequences([test], maxlen = seq_length, padding = 'post', value = char2idx['UNK'])

    pred = model.predict_classes(test)
    sentence += ' '+idx2char[pred[0]]
    #여기서 약간 실수가 있었는데 ' '가 아니라 ''으로 더했어야 나중에 자모 합칠때 단어가 된다
  print()
  print(jamotools.join_jamos(sentence))
  print()

test_cb = tf.keras.callbacks.LambdaCallback(test_cb)
model.fit(char_dataset.repeat(), steps_per_epoch=steps_per_epoch, callbacks=[test_cb], epochs = 7, verbose = 2)


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 80, 100)           619800    
_________________________________________________________________
lstm_10 (LSTM)               (None, 80, 100)           80400     
_________________________________________________________________
dropout_5 (Dropout)          (None, 80, 100)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 6198)              625998    
Total params: 1,406,598
Trainable params: 1,406,598
Non-trainable params: 0
_________________________________________________________________

﻿태조 이성계 선대의 가계. 목조 이안사가 전주에서 삼척·의주를 거쳐 알동에 정착하다  茇 茇 奪 枚 枚 奭 奭 奭 赴 赴 赴 赴 赴 赴 赴 赴 赴 赴 窩 韞 韞 涉 總 總 總 梗 梗 幄 幄 幄 

In [70]:
train_text[300:500]

'복야(僕射) 휘(諱) 이천상(李天祥)을 낳고, 복야가 아간(阿干) 휘(諱) 광희(光禧)를 낳고, 아간이 사도(司徒) 삼중 대광(三重大匡) 휘(諱) 입전(立全)을 낳고, 사도가 휘(諱) 이긍휴(李兢休)를 낳고, 이긍휴가 휘(諱) 염순(廉順)을 낳고, 염순이 휘(諱) 이승삭(李承朔)을 낳고, 이승삭이 휘(諱) 충경(充慶)을 낳고, 충경이 휘(諱) 경영(景英)을'

In [73]:
sentence = train_text[300:400]
sentence = jamotools.split_syllables(sentence)
for _ in range(300):
  test = sentence[-seq_length:]
  test = [char2idx[i] if i in char2idx else char2idx['UNK']for i in test]
  test = pad_sequences([test], maxlen = seq_length, padding = 'post', value = char2idx['UNK'])
  pred = model.predict_classes(test)
  #pred는 숫자로, 즉 예측하는 단어의 index를 출력
  sentence += ''+idx2char[pred[0]]
print(jamotools.join_jamos(sentence))

복야(僕射) 휘(諱) 이천상(李天祥)을 낳고, 복야가 아간(阿干) 휘(諱) 광희(光禧)를 낳고, 아간이 사도(司徒) 삼중 대광(三重大匡) 휘(諱) 입전(立全)을 낳고, 사도가 휘(李)를 정상(李山)·이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 이름이 일


- 시간이 너무 오래 걸려서 7번만 학습했지만 학습수를 높이면 충분히 학습이 가능할 것이다.
- RNN을 이용할떄는 연속으로 변화가 있어야 함에 주의해야 한다